# Gerenciador de envio de Pacotes para o Arduíno

## Bibliotecas

In [17]:
from serial import Serial
from time import sleep

## Comunicação serial com o python

In [ ]:
# use arduino/pyserial-echo.ino

with Serial(port = "/dev/ttyUSB0", baudrate = 9600, timeout = 5) as arduino:
	print("Estabilizando a comunicação")
	sleep(3)
	print("Comunicação estabilizada")
	
	encodedMessage = input().encode()
	bytesWritten = arduino.write(encodedMessage)

	print(f"Bytes escritos: {bytesWritten}")

	readBufferSize = 2**10
	bytesRead = arduino.read(readBufferSize)

	decodedBytes = bytesRead.decode()

	print(decodedBytes)


## Envio dos pacotes para o Arduino

In [ ]:
# use arduino/lazy-package.ino

readBufferSize = 2 ** 10

with Serial(port = "/dev/ttyUSB0", baudrate = 9600, timeout = 3) as arduino:
	print("Estabilizando a comunicação")
	sleep(3)
	print("Comunicação estabilizada")

	with open("flor.iot") as file:
		for line in file:

			while True:
				response = arduino.readline().decode().strip()
				print(f"Arduino: {response}")	

				if response == "READY":
					break

			command = line.strip() + "\n"
			commandEncoded = command.encode()

			print(f"PC: {command}")
			bytesWritten = arduino.write(commandEncoded)



## Classe do processo de impressão

In [21]:
class PlotterProcess:
	def __init__(self, port: str, filename: str):
		self.port = port
		self.filename = filename

		self.bufferSize = 1024 
		self.baudrate = 9600
		self.timeout = 3

	def start(self):
		with Serial(
				port = self.port, 
				baudrate = self.baudrate, 
				timeout = self.timeout
		) as arduino:
			print("[PC] Estabilizando a comunicação Serial")
			sleep(3)
			print("[PC] Comunicação estabilizada")

			with open(self.filename) as file:
				for line in file:

					while True:
						response = arduino.readline().decode().strip()
						print(f"[Arduino] {response}")	

						if response == "READY":
							break

					command = line.strip() + "\n"
					commandEncoded = command.encode()

					print(f"[PC] {command}")
					arduino.write(commandEncoded)

In [ ]:
process = PlotterProcess(
	port = "/dev/ttyUSB0",
	filename = "flor.iot"
)

process.start()